In [1]:
import geehydro
from rasterio.windows import from_bounds
import rasterio as rio
from rasterio.enums import Resampling
import os
from os.path import join, basename
import numpy as np
import pandas as pd
#import rioxarray
import matplotlib.pyplot as plt
import rasterio as rio
#from osgeo import gdal
from rasterio.windows import Window , from_bounds, crop
import folium
import pyproj
from glob import glob
import sys
from scipy import sparse

In [2]:
import ee
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AX4XfWhcecK1WerybgzFM6XWJjE8lE82WV5zprBOSaSF2ZZAwO7UfD5d0jk

Successfully saved authorization token.


In [3]:
home_dir = '/home/zacharykeskinen/Documents/uavsar/'
data_dir = join(home_dir,'data')
url_dir = join(data_dir, 'urls')
imgs_dir = join(home_dir, 'imgs')
loc_dir = (join(imgs_dir, os.listdir(imgs_dir)[0]))
auc_dir = join(data_dir, 'aucillary')
low_auc_dir = join(auc_dir, 'lowman')

In [4]:
####From: http://www2.geog.ucl.ac.uk/~plewis/geogg122-2011-12/dem2.html

def gaussianFilter(sizex,sizey=None,scale=0.333):
    '''
    Generate and return a 2D Gaussian function
    of dimensions (sizex,sizey)

    If sizey is not set, it defaults to sizex
    A scale can be defined to widen the function (default = 0.333)
    '''
    sizey = sizey or sizex
    x, y = np.mgrid[-sizex:sizex+1, -sizey:sizey+1]
    g = np.exp(-scale*(x**2/float(sizex)+y**2/float(sizey)))
    return g/g.sum()

def grad2d(dem):
    '''
    Calculate the slope and gradient of a DEM
    '''
    from scipy import signal
    f0 = gaussianFilter(3)
    I = signal.convolve(dem,f0,mode='valid')
    f1 = np.array([[-1,0,1],[-2,0,2],[-1,0,1]])
    f2 = f1.transpose()
    g1 = signal.convolve(I,f1,mode='valid')
    g2 = signal.convolve(I,f2,mode='valid')
    slope = np.sqrt(g1**2 + g2**2)
    aspect = np.arctan2(g2,g1)
    return slope, aspect

###convert to dB
def lin_to_dB(arr):
    return 10*np.log10(arr)

from scipy import interpolate
def array_resize(arr, target_shape, kind = 'linear'):

    H, W = arr.shape[:2]
    new_H, new_W = target_shape[:2]
    xrange = lambda x: np.linspace(0, 1, x)

    f = interpolate.interp2d(xrange(W), xrange(H), arr, kind=kind)
    new_arr = f(xrange(new_W), xrange(new_H))
    return new_arr

In [5]:
left = -115.6
right = -115.2
top = 44.16
bottom = 44.0

In [10]:
site = ee.Geometry.Polygon(
    [[[right, bottom],
      [left, bottom],
      [left, top],
      [right, top]]])

Map = folium.Map(location=[bottom,left],
                   tiles='Stamen Terrain', zoom_start = 11,control_scale = True)

# To see a google satellite view as a basemap
Map.setOptions('HYBRID')

Map.addLayer(site)

Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)

# Display map 
Map

In [11]:
Map.save('../figures/sub_square.html')

In [7]:
import xarray as xr

sub = xr.Dataset()
for im in glob(loc_dir + '/*.tif'):
    print(im)
    i=0
    fp = join(loc_dir,im)
    if '.inc' in im:
        name = '-'.join(basename(im).replace('.tif','').split('.')[1:])
    elif '.int.' in im:
        name = '-'.join(basename(im).replace('.tif','').split('.')[1:])
        pol = basename(im).split('.')[0].split('_')[-2][4:]
        name = pol+'-'+name
    else:
        name = '-'.join(basename(im).replace('.tif','').split('.')[1:2])
        pol = basename(im).split('.')[0].split('_')[-2][4:]
        name = pol+'-'+name
    print(name)
    if 'inc' in fp:
        with rio.open(fp) as src:
            my_proj = pyproj.Proj(str(src.crs).lower())#your data crs
            left_bottom = my_proj(left,bottom, inverse=False)
            right_top = my_proj(right,top, inverse=False)
            win = from_bounds(left_bottom[0], left_bottom[1], right_top[0], right_top[1], src.transform)

            arr = src.read(
            out_shape=(src.count,target_shape[0],target_shape[1]),
            resampling=Resampling.bilinear,
            window = win)[0]
        sub[name] = (('x','y'),arr)
    elif '.hgt.' in fp:
        if 'VV' in fp:
            with rio.open(fp) as src:
                arr = src.read(1, window = win)
                sub['dem'] = (('x','y'),arr)
                slope,aspect = grad2d(arr)
                slope = array_resize(slope, target_shape)
                aspect = array_resize(aspect, target_shape)
                sub['slope'] = (('x','y'),slope)
                sub['aspect'] = (('x','y'),aspect)
            
    else:
        with rio.open(fp) as src:
            if i == 0:
                my_proj = pyproj.Proj(str(src.crs).lower())#your data crs
                left_bottom = my_proj(left,bottom, inverse=False)
                right_top = my_proj(right,top, inverse=False)
                win = from_bounds(left_bottom[0], left_bottom[1], right_top[0], right_top[1], src.transform)
                target_shape = src.read(1, window = win).shape
                i = 1
            sub[name] = (('x','y'),src.read(1, window = win))

/home/zacharykeskinen/Documents/uavsar/imgs/lowman_test/lowman_05208_21019-019_21021-007_0006d_s01_L090HH_01.hgt.real.tif
HH-hgt
/home/zacharykeskinen/Documents/uavsar/imgs/lowman_test/lowman_05208_21019-019_21021-007_0006d_s01_L090VV_01.int.imaginary.tif
VV-int-imaginary
/home/zacharykeskinen/Documents/uavsar/imgs/lowman_test/lowman_05208_21019_019_210316_L090_CX_01.inc.tif
inc
/home/zacharykeskinen/Documents/uavsar/imgs/lowman_test/lowman_05208_21019-019_21021-007_0006d_s01_L090HV_01.int.real.tif
HV-int-real
/home/zacharykeskinen/Documents/uavsar/imgs/lowman_test/lowman_05208_21019-019_21021-007_0006d_s01_L090HH_01.int.imaginary.tif
HH-int-imaginary
/home/zacharykeskinen/Documents/uavsar/imgs/lowman_test/lowman_05208_21019-019_21021-007_0006d_s01_L090HH_01.amp2.real.tif
HH-amp2
/home/zacharykeskinen/Documents/uavsar/imgs/lowman_test/lowman_05208_21019-019_21021-007_0006d_s01_L090HH_01.amp1.real.tif
HH-amp1
/home/zacharykeskinen/Documents/uavsar/imgs/lowman_test/lowman_05208_21019-019

In [8]:
for im in glob(low_auc_dir + '/*resamp.tif', recursive = True):
    with rio.open(im) as src:
        dtype = basename(im).replace('_resamp.tif','')
        pol = None
        real_img = 'real'
        sub[dtype] = (('x','y'),src.read(1, window = win))

In [9]:
for img in os.listdir(loc_dir):
    if 'csv' in img:
        desc = pd.read_csv(join(loc_dir, img))

for col in desc.columns:
    if col not in desc.columns[0]:
        if desc.loc[1,col] != '&':
            sub.attrs[col] =  str(desc.loc[0,col])+ ' ' +str(desc.loc[1,col])
        else:    
            sub.attrs[col] =  str(desc.loc[0,col])
sub.attrs['subset_coords'] = [left,bottom,right,top]

In [10]:
site = ee.Geometry.Polygon(
    [[[right, bottom],
      [left, bottom],
      [left, top],
      [right, top]]])

from datetime import datetime
start_date = pd.to_datetime(desc['start time of acquisition for pass 1'][0])
end_date = pd.to_datetime(desc['start time of acquisition for pass 2'][0])
start_date = f'{start_date.year}-{start_date.month}-{start_date.day}'
end_date = f'{end_date.year}-{end_date.month}-{end_date.day}'
print(start_date)

prism = ee.ImageCollection('OREGONSTATE/PRISM/AN81d').filter(ee.Filter.date(start_date, end_date));
total_ppt = prism.select('ppt').sum()
mean_temp = prism.select('tmean').mean()

def freezing_days(image):
    return image.updateMask(image.gt(0))

days_mean_af = prism.select('tmean').map(freezing_days).reduce(ee.Reducer.count())
days_max_af = prism.select('tmax').map(freezing_days).reduce(ee.Reducer.count())

2021-3-16


In [11]:
Map = folium.Map(location=[bottom,left],
                   tiles='Stamen Terrain', zoom_start = 13)

# To see a google satellite view as a basemap
Map.setOptions('HYBRID')

Map.addLayer(prism.select('tmean').mean().clip(site), {min:0,max:3}, 'tmean', 1)
Map.addLayer(total_ppt.clip(site), {'min':0,'max':50}, 'tree coverage', 0)
Map.addLayer(mean_temp.clip(site), {'min':0,'max':5}, 'ground coverage', 0)
Map.addLayer(days_mean_af.clip(site), {'min':0, 'max':6}, 'NED land forms', 0)
Map.addLayer(days_max_af.clip(site), {'min':0,'max':8}, 'Lithography', 0)

Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)

# Display map 
Map

In [12]:
imdic = {total_ppt:'ppt',mean_temp:'tmean',days_mean_af: 'tmean_count', days_max_af: 'tmax_count'}
namedic = {total_ppt:'ppt_total',mean_temp:'tmean',days_mean_af: 'mean_af_days', days_max_af: 'max_af_days'}
for im in [total_ppt, mean_temp, days_mean_af, days_max_af]:
    layer = imdic[im]
    name = namedic[im]
    arr = np.array(im.select(layer).reproject(scale = 1000,crs = mean_temp.projection()).sampleRectangle(site).get(layer).getInfo())
    new_arr = array_resize(arr, target_shape)
    sub[name] = (('x','y'),new_arr)

In [13]:
from scipy import ndimage
from tqdm import tqdm

KERNEL_DIMENSIONS=[5,5] # doesn't have to be square

#Set up function 
def focal_variance(subarr):
	"""Calcualte variance of an array
	"""
	return(ndimage.variance(subarr))

#Set up moving window (kernel) dimensions
footprint=np.ones((KERNEL_DIMENSIONS[0],KERNEL_DIMENSIONS[1])) 

#Apply function across dataset
processedArray= ndimage.generic_filter(sub['dem'], focal_variance, footprint=footprint)


In [14]:
processedArray = array_resize(processedArray, target_shape)

In [15]:
sub['focal_var'] = (('x','y'),processedArray)

In [ ]:
import rasterio
import numpy as np
from affine import Affine
from pyproj import Proj, transform
i = 0
for im in glob(loc_dir + '/*.tif'):
    if i == 0:
        with rasterio.open(im) as src:
            band1 = src.read(1)
            print('Band1 has shape', band1.shape)
            height = band1.shape[0]
            width = band1.shape[1]
            cols, rows = np.meshgrid(np.arange(width), np.arange(height))
            xs, ys = rasterio.transform.xy(src.transform, rows, cols)
            lons= np.array(xs)
            lats = np.array(ys)
            print('lons shape', lons.shape)
    i = 1    
print(A.shape)

Band1 has shape (21603, 23940)


In [ ]:
for name, values in sub.items():
    if name != 'dem':
        sub[name] = sub[name].where(sub['dem'] != -10000.0)
sub['dem'] = sub['dem'].where(sub['dem'] != -10000.0)

In [ ]:
sub

In [ ]:
sub.to_netcdf('../data/netcdfs/banner_lowman_05208_21019-019_21021-007_0006d.nc')